In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, Conv1D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
from tqdm import tqdm
from scipy.io.wavfile import write

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
img_rows = 128
img_cols = 128
img_shape = (img_rows, img_cols)
latent_dim = 100
_min = 0
_max = 3140.9622

In [ ]:
X_train = np.load('drum-data.npy')

In [ ]:
def build_generator():
    model = Sequential()
    
    # layer output size (16, 1024)
    model.add(Dense(16 * 64 * 16, activation='relu', input_dim=latent_dim))
    model.add(Reshape((16, 1024)))
    model.add(Activation('relu'))
    
    # layer output size (64, 512)
    model.add(Reshape((1, 16, 1024)))
    model.add(Conv2DTranspose(512, kernel_size=(1, 25), strides=(1, 4), padding='same'))
    model.add(Activation('relu'))

    # layer output size (256, 256)
    model.add(Conv2DTranspose(256, kernel_size=(1, 25), strides=(1, 4), padding='same'))
    model.add(Activation('relu'))
    
    # layer output size (1024, 128)
    model.add(Conv2DTranspose(128, kernel_size=(1, 25), strides=(1, 4), padding='same'))
    model.add(Activation('relu'))
    
    # layer output size (4096, 64)
    model.add(Conv2DTranspose(64, kernel_size=(1, 25), strides=(1, 4), padding='same'))
    model.add(Activation('relu'))
    
    # layer output size (128, 128)
    model.add(Conv2DTranspose(1, kernel_size=(1, 25), strides=(1, 4), padding='same'))
    model.add(Reshape((16384, 1)))
    model.add(Activation('tanh'))
    model.add(Reshape((128, 128)))

    return model

In [ ]:
def build_discriminator():
    model = Sequential()
    
    model.add(Conv1D(64, kernel_size=25, strides=4, input_shape=img_shape, padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv1D(128, kernel_size=25, strides=4, padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv1D(256, kernel_size=25, strides=4, padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv1D(512, kernel_size=25, strides=4, padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv1D(1024, kernel_size=25, strides=4, padding='same'))
    model.add(LeakyReLU())
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
def train(X_train, gan_model, discriminator, epochs, batch_size=128, save_interval=100, progress_interval=1):
    # prepare target outpus
    ones = np.ones((batch_size, 1))
    zeros = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select random images
        index = np.random.randint(0, X_train.shape[0], batch_size)
        real_images = X_train[index]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_images = generator.predict(noise)
        
        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(real_images, ones)
        d_loss_fake = discriminator.train_on_batch(generated_images, zeros)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator 
        g_loss = gan_model.train_on_batch(noise, ones)

        if epoch % progress_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        if epoch % save_interval == 0:
            save_images(epoch)
            
        #if epoch % (save_interval * 2) == 0:
            #save_audio(epoch)
            
    return gan_model, discriminator

In [ ]:
def save_images(epoch):
    rows, cols = 2, 2
    noise = np.random.normal(0, 1, (rows * cols, latent_dim))
    gen_imgs = generator.predict(noise)

    fig, axs = plt.subplots(rows, cols)
    cnt = 0
    for i in range(rows):
        for j in range(cols):
            axs[i,j].imshow(gen_imgs[cnt, :,:], cmap='hot')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/drums_%d.png" % epoch)
    plt.close()

In [ ]:
def save_audio(epoch):
    noise = np.random.normal(0, 1, (1, latent_dim))
    generated_images = generator.predict(noise)
    generated_image = np.squeeze(generated_images[0])
    audio = librosa.feature.inverse.mel_to_audio(denormalize(generated_image), sr=22050)
    write('audio/drums_' + str(epoch) + '.wav', 22050, audio)

In [ ]:
def denormalize(array):
    return (array * (_max - _min)) + _min

In [ ]:
generator = build_generator()
discriminator = build_discriminator()
d_optimizer = Adam(lr=0.0001)
discriminator.compile(loss='binary_crossentropy', optimizer=d_optimizer, metrics=['accuracy'])
discriminator.trainable = False
gan_model = Sequential()
gan_model.add(generator)
gan_model.add(discriminator)
gan_optimizer = Adam(lr=0.0001)
gan_model.compile(loss='binary_crossentropy', optimizer=gan_optimizer)

In [ ]:
gan_model, discriminator = train(X_train, gan_model, discriminator, epochs=1501, batch_size=32, save_interval=250, progress_interval=50)

In [ ]:
noise = np.random.normal(0, 1, (1, latent_dim))
generated_images = generator.predict(noise)
generated_image = np.squeeze(generated_images)
plt.imshow(generated_image, cmap='hot')

test = librosa.feature.inverse.mel_to_audio(denormalize(generated_image), sr=22050)
write('test.wav', 22050, test)